# First Project - Food2Vec
Machine Learning for NLP

DIA1 - Romain GIRODET et Annabel MERCERON

WARNING: This document is ~7 minuts to execute. 

Link to the problem : https://devinci-online.brightspace.com/d2l/le/lessons/90360/topics/281988

Link to the dataset : https://fr.openfoodfacts.org/data

The dataset we were given has a list of products, their ingredients, and a series of other columns. The objective of the project is to draw a 2D map of the products according to their ingredients.

We also chose to go further and exploit the different possibilities available to us, after having vectorized these lists of ingredients.

First, we imported a piece of the csv so that we could observe, clean the data, do some initial experiments, etc. The first part of our code is NOT TO BE EXECUTED because of its extremely long execution time (import of the csv, translation...)

## Imports

### Librairies

In [53]:
#pip install googletrans

In [54]:
import pandas as pd
import numpy as np
from math import *
import nltk

from nltk import pos_tag, word_tokenize
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import spatial

from sklearn import decomposition, cluster
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

import warnings

import string 

from googletrans import Translator

In [55]:
import socket
socket.getaddrinfo('localhost', 8080)

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)


### Data

In [56]:
df = pd.read_csv("Food2Vec.csv")
df.head()

C:\Users\julia\AppData\Local\Temp\ipykernel_16272\166335402.py:1: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.



,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,brands,brands_tags,...,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,calcium_100g,iron_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,nutrition-score-fr_100g,ratio
0,00000000000026772226,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1654250311,2022-06-03T09:58:31Z,1654270474,2022-06-03T15:34:34Z,Skyr,Danone,danone,...,3.9,NaN,10.0,0.09,0.036,NaN,NaN,NaN,-5.0,0.274194
1,0000000000100,http://world-en.openfoodfacts.org/product/0000...,del51,1444572561,2015-10-11T14:09:21Z,1444659212,2015-10-12T14:13:32Z,moutarde au moût de raisin,courte paille,courte-paille,...,22.0,0.0,5.1,4.60,1.840,NaN,NaN,0.0,18.0,0.193548
2,0000000001281,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1517830801,2018-02-05T11:40:01Z,1644567888,2022-02-11T08:24:48Z,Tarte noix de coco,"Crous Resto',Crous","crous-resto,crous",...,21.9,4.4,4.6,0.10,0.040,NaN,NaN,NaN,14.0,0.258065
3,0000000001885,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1511180337,2017-11-20T12:18:57Z,1518126491,2018-02-08T21:48:11Z,Compote de poire,Crous,crous,...,27.0,3.6,0.6,0.00,0.000,NaN,NaN,NaN,-2.0,0.258065
4,0000000002257,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1520506122,2018-03-08T10:48:42Z,1561463749,2019-06-25T11:55:49Z,Salade de macedoine de légumes,NaN,NaN,...,1.0,NaN,1.9,0.27,0.108,NaN,NaN,NaN,1.0,0.290323


## Data cleaning

In [57]:
#rename des colonnes
df=df.rename(columns = {'ingredients_tags':'ingredient', 'product_name':'produit'})
#seules colonnes qui nous intéressent
df=df[["produit","ingredient"]]
#suppression des valeurs nulles dans ingredient
df=df.dropna(axis=0,subset='ingredient')
#maj des indexes
df.index=[i for i in range(len(df))]

df

,produit,ingredient
0,moutarde au moût de raisin,fr:eau-graines-de-teguments-de-moutarde-vinaig...
1,BAguette bressan,"fr:baguette-bressan-pain-baguette,en:wheat-flo..."
2,Naturablue original,"en:e422,en:spirulina,fr:extrait-de-fruits-de-c..."
3,Pâte a tartiner,"fr:noisettes-locales,en:cane-sugar,en:added-su..."
4,Organic Baby Lima Beans,"en:baby-lima-beans,en:legume,en:lima-bean"
...,...,...
54809,Zucchini & summer squash noodles,"en:courgette,en:vegetable,en:yellow-summer-squash"
54810,"Whole foods market, southwest sweet potato salad","en:sweet-potato,en:vegetable,en:root-vegetable..."
54811,Miso glazed carrots,"en:carrot,en:vegetable,en:root-vegetable,en:ci..."
54812,Buffalo cauliflower,"en:cauliflower,en:vegetable,en:cindy-s-hot-sau..."


## Formatting the column "ingredient"

We chose to execute our code on 2.000 data for 2 main reasons :
- the execution time is quite long (for 1.000 data : 7min, for 2.000 data : 40min)
- to view the 2D map, we can see the points and the products better with 1000 products
- the google API for traduction doesn't allow to go beyond (too many requests in too short a time) 

However, you can change the number of data taken.

In [58]:
df_cleaned=df[:1000].copy()

### Traduction and assembling

In [59]:
def traduction_Assemblage(element):

    dico={"fr":[],"en":[],"esp":[],"all":[]}
    dico_join={"fr":[],"en":[],"esp":[],"all":[]}

    char_to_replace={"-":"","fr:":"en:","esp":"","all":""}

    for e in element.split(","):
        if e[:3]=="fr:":
            dico["fr"].append(e.replace("fr:",""))
        elif e[:3]=="en:": 
            dico["en"].append(e.replace("en:",""))
        elif e[:3]=="esp:": 
            dico["esp"].append(e.replace("esp:",""))
        elif e[:3]=="all:": 
            dico["all"].append(e.replace("all:",""))

    for key,list in dico.items():
        if list!=[]:
            dico_join[key]=(" ".join(list)).replace("-"," ")

    tr = Translator()

    if dico_join["fr"]!=[]:
        dico_join["fr"]=tr.translate(dico_join["fr"],src='fr',dest='en').text
    if dico_join["esp"]!=[]:
        dico_join["esp"]=tr.translate(dico_join["esp"],src='es',dest='en').text
    if dico_join["all"]!=[]:
        dico_join["all"]=tr.translate(dico_join["all"],src='de',dest='en').text

    total_mot=""
    for value in dico_join.values():
        if value !=[]:
           
            total_mot+=" "+value

    return total_mot[1:]

In [60]:
df_cleaned["ingredients"]=df_cleaned["ingredient"].apply(lambda x:(traduction_Assemblage(x)))
df_cleaned.drop("ingredient",axis=1,inplace=True)
df_cleaned.head()

,produit,ingredients
0,moutarde au moût de raisin,water mustard seed coats red wine vinegar salt...
1,BAguette bressan,baguette bressan baguette bread dextrose gluco...
2,Naturablue original,camu camu fruit extract spirulina maxima myrci...
3,Pâte a tartiner,local hazelnuts bitter cocoa grapeseed oil 82 ...
4,Organic Baby Lima Beans,baby lima beans legume lima bean


### Remove errors in the writing of ingredients

In [61]:
def remove_punctuation(x):
    for punctuation in string.punctuation:
        x=x.replace(punctuation,"")
    return x

def remove_letter(x):
    return " ".join([i for i in x.split(" ") if i not in string.ascii_lowercase])


In [62]:
df_cleaned["ingredients"]=df_cleaned["ingredients"].apply(lambda x:(remove_punctuation(x)))
df_cleaned["ingredients"]=df_cleaned["ingredients"].apply(lambda x:(remove_letter(x)))
df_cleaned.head()

,produit,ingredients
0,moutarde au moût de raisin,water mustard seed coats red wine vinegar salt...
1,BAguette bressan,baguette bressan baguette bread dextrose gluco...
2,Naturablue original,camu camu fruit extract spirulina maxima myrci...
3,Pâte a tartiner,local hazelnuts bitter cocoa grapeseed oil 82 ...
4,Organic Baby Lima Beans,baby lima beans legume lima bean


### Cutting the word list into groups of words according to meaning

In [63]:
def repartitionMots(listIngr):
    """Fonction permettant de couper une liste de mots ("arôme naturel de citron vert arôme stabilisateur naturel") en groupes de mots ("arôme naturel de citron vert", "arôme stabilisateur naturel")"""
    
    listTAG = nltk.pos_tag(nltk.word_tokenize(listIngr))

    #mot liaison : vont avec le mot précédent et le suivant ; exemple : OF dans "pair OF shoes"
    liaison = ["CC", "DT", "IN", "RB", "RBR", "VBD", "VBG", "VB", "VBN", "VBP", "VBZ"]
    #mot simple : vont seul ou avec le précédent si précisé ; exemple : "shoe"
    #mot = ["NN", "NNS", "NNP", "NNPS", "SYM"]
    #mot complément : vont avec le mot suivant ; exemple : "ORANGE shoe" ou "A shoe"
    complement=["CD", "EX", "JJ", "JJR", "JJS", "PP", "POS", "PRP", "RP", "WP", "WP", "WRB"]


    listRetour = []
    link = False  # booléen pour indiquer si le mot précédent est un mot de liaison. Si True : le mot suivant va avec le précédent
    for i in range(len(listTAG)):
        if i == 0:  # si c'est le premier elt de la liste on le met directement dans la liste finale
            nouvel_element = [listTAG[i][0]]
            listRetour.append(nouvel_element)
            #indexRETOUR=1

        elif any(chr.isdigit() for chr in listTAG[i][0]) :#si le mot contient un chiffre : les additifs e422
            nouvel_element = [listTAG[i][0]]
            listRetour.append(nouvel_element)
            link = False
        
        elif listTAG[i][0].find('a')==len(listTAG[i][0])-1: #si un string finit par un "a", c'est un mot scientifique
            nouvel_element = [listTAG[i][0]]
            listRetour.append(nouvel_element)
            link = False


        elif link == True:  # si le mot est précédé d'un mot de liaison, on l'ajoute au mot précédent
            listRetour[len(listRetour)-1].append(listTAG[i][0])
            if listTAG[i][1] not in complement :
                link = False

        else:  # link==False ; si le mot n'est pas précédé d'un mot de liaison

            # si l'élt est un adjectif, on l'intègre à l'élément précédent
            if listTAG[i][1] in complement:
                nouvel_element = [listTAG[i][0]]
                listRetour.append(nouvel_element)
                link = True

            # si l'élt est un mot de liaison, on l'intègre à l'élément précédent et on indique que le mot suivant sera aussi intégré
            elif listTAG[i][1] in liaison:
                listRetour[len(listRetour)-1].append(listTAG[i][0])
                link = True
            
            else : #elif listTAG[i][1] in mot:  # si l'élt est un nom et qu'il n'est pas précédé d'un adv
                nouvel_element = [listTAG[i][0]]
                listRetour.append(nouvel_element)  # on le met à la suite
    
    phraseRetour = []
    for i in listRetour:
        phrase=""
        for j in range(len(i)):
            if j==0:
                phrase+=i[0]
            else:
                phrase+=" "+i[j]
        phraseRetour.append(phrase)

    return phraseRetour

In [64]:
df_cleaned["ingredients"]=df_cleaned["ingredients"].apply(lambda x:(repartitionMots(str(x))))
df_cleaned.head()

,produit,ingredients
0,moutarde au moût de raisin,"[water, mustard, seed, coats, red wine, vinega..."
1,BAguette bressan,"[baguette, bressan, baguette, bread dextrose g..."
2,Naturablue original,"[camu, camu, fruit, extract, spirulina, maxima..."
3,Pâte a tartiner,"[local, hazelnuts, bitter, cocoa grapeseed oil..."
4,Organic Baby Lima Beans,"[baby, lima, beans legume, lima, bean]"


### Remove errors in the writing of the column "ingredient"

In [65]:
def remove_ingredient_sup_5(list_ingredients):
    return [i for i in list_ingredients if len(i.split(" "))<=4]

def remove_digit(list_ingredients):
    return [i for i in list_ingredients if not(i.isdigit())]

def remove_it(list_ingredients):
    return [i for i in list_ingredients if "it" not in i]

def remove_not_food(list_ingredients):
    to_remove=["ingredients", "ingredient"]
    return [i for i in list_ingredients if i not in to_remove ]

def good_product_format(produit):
    return str(produit).lower().strip()

In [66]:
df_cleaned["ingredients"]=df_cleaned["ingredients"].apply(lambda x:(remove_ingredient_sup_5(x)))
df_cleaned["ingredients"]=df_cleaned["ingredients"].apply(lambda x:(remove_digit(x)))

df_cleaned["ingredients"]=df_cleaned["ingredients"].apply(lambda x:(remove_it(x)))
df_cleaned["ingredients"]=df_cleaned["ingredients"].apply(lambda x:(remove_not_food(x)))

df_cleaned["produit"]=df_cleaned["produit"].apply(lambda x:(good_product_format(x)))
df_cleaned

,produit,ingredients
0,moutarde au moût de raisin,"[water, mustard, seed, coats, red wine, vinega..."
1,baguette bressan,"[baguette, bressan, baguette, bread dextrose g..."
2,naturablue original,"[camu, camu, extract, spirulina, maxima, myrci..."
3,pâte a tartiner,"[local, hazelnuts, cocoa grapeseed oil, sugar,..."
4,organic baby lima beans,"[baby, lima, beans legume, lima, bean]"
...,...,...
9995,pineapple tidbits in pineapple juice from conc...,"[pineapple, sectors, water, pineapple, juice f..."
9996,"spartan, pasta salad, ranch & bacon","[pasta, dough, salt, whey, dairy, e621, natura..."
9997,"pasta salad, creamy italian pasta, vegetable b...","[pasta, dough, disaccharide, garlic, vegetable..."
9998,pasta salad,"[pasta dough wheat, flour cereal, flour wheat,..."


## 2-dimensional visualization of the products according to the ingredients

In [67]:
def __Create_OneHot_Colonnes__(x, name):
    if name in x["ingredients"]:
        return 1
    else:
        return 0

In [68]:
def __Create_dfOneHot__(df):
    df_OneHot=df.copy()
    liste_ingredients=[]

    #collecte de tous les ingredients dans une list : liste_ingredients
    for i in range(len(df_OneHot)):
        liste_ingredients+=df_OneHot.loc[i,"ingredients"]

    #suppression des ingredienst doublons : liste_ingredients_uniques 

    liste_ingredients_uniques=np.unique(liste_ingredients).tolist()
    

    #Creation des colonnes pour one hot encoding 
    for ingredient in liste_ingredients_uniques:
        df_OneHot[ingredient]=0
    df_OneHot
    
    #One Hot Encoding  
    for name in liste_ingredients_uniques:
        df_OneHot[name] = df_OneHot.apply(lambda x: __Create_OneHot_Colonnes__(x, name), axis=1)
    
    
    df_OneHot.drop(labels=["ingredients"],axis=1,inplace=True)

    return (df_OneHot,liste_ingredients_uniques)

In [69]:
def Visualisation_2D_Dataframe_K_Cluster (df_OneHot,df, nb_cluster):

  

    X = df_OneHot[df_OneHot.columns[df_OneHot.columns!="produit"]]
    y = df_OneHot[df_OneHot.columns[df_OneHot.columns=="produit"]]

    # PCA 2 dimension 
    pca = PCA(n_components=2)
    components=pca.fit_transform(X)
    # Create 3 col dataframe : produits | components 0 | components 1
    df_2dim=pd.DataFrame(components,columns=["components_0","component_1"])
    df_2dim["produit"]=y
    df_2dim

    # Ajout 4ème colonne label à l'aide du k-means algorithme 

    #clustering =DBSCAN(eps=0.01).fit(X)
    clustering =KMeans(n_clusters=nb_cluster).fit(X)
    df_2dim["label"]=clustering.labels_

    df_2dim=df_2dim.merge(right=df,how="inner")
    
    
    fig = px.scatter(df_2dim, x=df_2dim["components_0"], y=df_2dim["component_1"], color=df_2dim["label"],hover_name=df_2dim["produit"])
    fig.show()

    return (df_2dim,df_OneHot)

In [70]:
# We choose to select number of cluster = len(df_cleaned)/50
n_cluster=int(len(df_cleaned)/50)
(df_OneHot,ingredients)=__Create_dfOneHot__(df_cleaned)
df_OneHot.to_csv("df_OneHot_10000.csv",index=False)


In [ ]:
(df_2dim,df_OneHot) = Visualisation_2D_Dataframe_K_Cluster(df_OneHot,df_cleaned, n_cluster)
df_OneHot = df_OneHot.merge(df_2dim,how="left",on="produit")

Note: the colors were assigned by the KMean function. We therefore cannot know how the labels were awarded. However, foods are spatially organized according to their consistency of composition.

## Exploration of the new possibilities following the vectorization of the products

The Euclidean distance function has been very useful for the following functions, especially to determine the nearest products.

In [ ]:
def DistanceEuclidienne(x,y):
    dist= pow(x["components_0"]-y["components_0"],2)+pow(x["component_1"]-y["component_1"],2)
    return dist

### Give the k closest products to a given product

This function takes a product as argument, and returns the k closest products (in terms of food composition) to the entered product.

In [ ]:
def kNeighbors(produit, df, k):    
    dataset=df.copy()
    
    #print(dataset)
    #La fonction ne commence que si le produit entré existe
    if produit in dataset['produit'].to_list():

        indexProduit = (dataset["produit"].to_list()).index(produit)
        y=dataset.loc[indexProduit]
        dataset.drop(index=indexProduit, inplace=True)# on vire le produit sinon on le comparera a lui même lors de la boucle
        
        #ajouter colonne dist euclidienne
        dataset["distance"] = dataset.apply(lambda x: DistanceEuclidienne(x,y), axis=1)
        #trier par ordre
        dataset.sort_values(by="distance",axis=0,inplace=True)
 
        return dataset[:k]["produit"]
    
    else :
        return "Ce produit n'existe pas"

In [ ]:
test=kNeighbors("baguette bressan",df_2dim,3)
test

682                        avocado
3668    valentine exchange dippers
413            rillettes de canard
Name: produit, dtype: object

### Give the k nearest products that contain a given food

In [ ]:
def k_produit_avec_aliment_LesPlusProches(produit,aliment,df_OneHot,k):
    df=df_OneHot.copy()

    if produit in df["produit"].tolist():
        print(produit,"exist")

        indexProduit = (df["produit"].to_list()).index(produit)
        y=df.loc[indexProduit]

        df.drop(index=indexProduit, inplace=True)# on vire le produit sinon on le comparera a lui même lors de la boucle


        if aliment in df_OneHot.columns.tolist():
            print(aliment,"exist")

            df=df[df[aliment]==1]
            df["distance"]=df.apply(lambda x: DistanceEuclidienne(x,y),axis=1)
            df.sort_values(by="distance",axis=0,inplace=True)
            
        else:
            print(aliment," don't exist")
    else:
        print(produit,"don't exist")
        
    return df["produit"][:k]


In [ ]:
k_produit_avec_aliment_LesPlusProches("moutarde au moût de raisin","water",df_OneHot,5)

moutarde au moût de raisin exist
water exist


C:\Users\julia\AppData\Local\Temp\ipykernel_16272\2708228576.py:25: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.



641                organic tomatoes, diced in tomato juice
2705     saranac, soft drink, black cherry cream, black...
322                       spray candy all kidz blend bonus
17698              caribbean rhythms juice, tropical mango
2416                       delicatessen style pepperoncini
Name: produit, dtype: object

### Give the products that contain a list of ingredients
This function can be useful for example if a person has ingredients in his fridge, wants to make a recipe from these ingredients, but has no idea. This function will then return recipes that contain these ingredients as well as additional ingredients to buy to complete the recipes.

In [ ]:
def match_produits_possedants_aliment(liste_ingredients,df_OneHot,k):
    df=df_OneHot.copy()

    for ingredient in liste_ingredients:

        if ingredient in ingredients:
            df=df[df[ingredient]==1]
        
    df=df[["produit","ingredients"]]

    df["ingredients manquants"]=df["ingredients"].apply(lambda x: [i for i in x if i not in liste_ingredients ])
    return df[:k]


In [ ]:
match_produits_possedants_aliment(["water","fruit"],df_OneHot,5)

,produit,ingredients,ingredients manquants
0,moutarde au moût de raisin,"[water, mustard, seed, coats, red wine, vinega...","[mustard, seed, coats, red wine, vinegar, salt..."
18,salade cesar,"[lemon, canola, oil, oil and fat, vegetable oi...","[lemon, canola, oil, oil and fat, vegetable oi..."
19,danoises à la cannelle roulées,"[tartaric esters of acetylated, flour baking, ...","[tartaric esters of acetylated, flour baking, ..."
30,root beer,"[carbonated, water, water, high fructose corn,...","[carbonated, high fructose corn, syrup added s..."
31,a&w - root beer - 355ml,"[high, fructose corn, syrup, preservative colo...","[high, fructose corn, syrup, preservative colo..."
